In [ ]:
# ===================================================================
# CELDA 1: Configuración Inicial, Imports y Funciones
# ===================================================================
import pandas as pd
import re
from hubspot import HubSpot
from hubspot.crm.properties import ApiException, PropertyUpdate, Option
from rut_chile import rut_chile as rc
import requests
import numpy as np
import unicodedata
from datetime import datetime, timedelta
import time
import psutil
import os
import concurrent.futures
import sys

# --- Archivo de Log ---
log_file = open("log.txt", "w")
log_file.write("====================================================\n")
log_file.write("== INICIO DEL SCRIPT: Nuevos Estado Pago ==\n")
log_file.write("====================================================\n")

start_time = time.time()
process = psutil.Process(os.getpid())
mem_start = process.memory_info().rss

# --- Funciones de Ayuda (Limpieza y Validación) ---
def validar_rut(rut):
    try:
        return rc.is_valid_rut(rut)
    except Exception:
        return False

def agregar_guion(valor):
    if pd.notna(valor) and isinstance(valor, str) and len(valor) > 1:
        return valor[:-1] + '-' + valor[-1]
    return valor

def eliminar_cero(valor):
    if isinstance(valor, str) and valor.startswith('0'):
        return valor[1:]
    return valor

def limpiar_valor(valor):
    if not isinstance(valor, str):
        return ""
    valor_sin_tildes = unicodedata.normalize('NFKD', valor).encode('ASCII', 'ignore').decode('ASCII')
    valor_limpio = re.sub(r'[^\w\s]', '', valor_sin_tildes)
    return valor_limpio.replace(' ', '_').lower()

log_file.write("Funciones de ayuda definidas.\n")

In [ ]:
# ===================================================================
# CELDA 2: Funciones de API Externa
# ===================================================================

def autorizacion_api():
    url_auth = 'https://authentication.colegiomedico.cl/api/authentication/authenticate'
    user = 'UsersCRSantiago' # Idealmente, esto también debería ser un secret
    psw = 'UsersCRSantiago2024_&_&' # Idealmente, esto también debería ser un secret
    
    data = {"userName": user, "password": psw}
    headers = {"Content-Type": "application/json"}
    
    try:
        response_auth = requests.post(url_auth, headers=headers, json=data, timeout=15)
        response_auth.raise_for_status()
        access_token = response_auth.text.strip('"')
        log_file.write("Token de autorización para API externa obtenido.\n")
        return access_token
    except requests.RequestException as e:
        log_file.write(f"!!! ERROR al autenticar con API externa: {e}\n")
        return None

def consulta_api(id, access_token, identificador='rut'):
    if identificador == 'rut':
        url = f'https://medicocontratosinfo.colegiomedico.cl/api/MedicoContratos/byRut/{id}/true'
    else:
        return None # Solo se maneja 'rut' en este script

    headers = {'Authorization': f'Bearer {access_token}'}
    try:
        response = requests.get(url, headers=headers, timeout=15)
        if response.status_code == 404:
            log_file.write(f"Info: RUT {id} no encontrado en API externa (404).\n")
            return None # No es un error, simplemente no hay datos
        response.raise_for_status()
        return response.json()
    except requests.RequestException as e:
        log_file.write(f"!!! ERROR al consultar ID {id} en API externa: {e}\n")
        return None

def obtener_datos_desde_api_rut(df_ids, columna_id, identificador='rut'):
    resultados = []
    total_ids = len(df_ids)
    access_token = autorizacion_api()
    if not access_token:
        log_file.write("No se pudo obtener token, abortando consulta a API externa.\n")
        return pd.DataFrame()

    for idx, id_actual in enumerate(df_ids[columna_id], start=1):
        respuesta = consulta_api(str(id_actual), access_token, identificador=identificador)
        if respuesta:
            resultados.append(respuesta)
        if idx % 50 == 0 or idx == total_ids:
            log_file.write(f"Progreso API externa: {idx}/{total_ids} IDs procesados.\n")
    return pd.DataFrame(resultados)

log_file.write("Funciones de API externa definidas.\n")

In [ ]:
# ===================================================================
# CELDA 3: Extracción y Procesamiento de Datos de HubSpot
# ===================================================================
log_file.write("\n--- Bloque 3: Extrayendo y procesando datos de HubSpot ---\n")
df_rut_filtrado_crm = pd.DataFrame()
try:
    api_client = HubSpot(access_token=os.getenv('HUBSPOT_API_KEY'))
    properties = [ 'firstname', 'lastname', 'createdate', 'rut', 'rcm', 'fecha_actualizacion_pagos', 'email', 'mobilephone', 'condicion_vital']
    log_file.write("Recuperando contactos de HubSpot...\n")
    all_contacts = api_client.crm.contacts.get_all(properties=properties)
    contacts_list = [contact.to_dict() for contact in all_contacts]
    log_file.write(f"Se recuperaron {len(contacts_list)} contactos.\n")

    df_contacts = pd.DataFrame(contacts_list)
    df_col_dict = pd.json_normalize(df_contacts['properties'])
    df_final_contactos = df_contacts.drop(columns=['properties']).join(df_col_dict)

    df_final_contactos['rut_normalizado'] = (
        df_final_contactos['rut'].astype(str).str.replace(r'[\s.,-]', '', regex=True).apply(agregar_guion).apply(eliminar_cero))
    df_final_contactos['rut_valido'] = df_final_contactos['rut_normalizado'].apply(validar_rut)
    df_final_contactos['rut_sdv'] = df_final_contactos['rut_normalizado'].str.split('-').str[0]

    df_rut_filtrado_crm = df_final_contactos[df_final_contactos['rut_valido']].copy()
    log_file.write(f"{len(df_rut_filtrado_crm)} contactos con RUT válido.\n")

except Exception as e:
    log_file.write(f"!!! ERROR en el bloque de HubSpot: {e}\n")

In [ ]:
# ===================================================================
# CELDA 4: Filtrado de Contactos para Actualización
# ===================================================================
df_ruts_a_consultar = pd.DataFrame()
if not df_rut_filtrado_crm.empty:
    log_file.write("\n--- Bloque 4: Filtrando contactos para actualizar ---\n")
    df_rut_filtrado_crm['fecha_actualizacion_pagos'] = pd.to_datetime(df_rut_filtrado_crm['fecha_actualizacion_pagos'], errors='coerce')
    df_rut_filtrado_crm = df_rut_filtrado_crm[(df_rut_filtrado_crm['condicion_vital'].isna()) | (df_rut_filtrado_crm['condicion_vital'] == 'VIVO')]
    df_filtrado_rut = df_rut_filtrado_crm[df_rut_filtrado_crm['fecha_actualizacion_pagos'].isna()].copy()
    df_ruts_a_consultar = df_filtrado_rut.sort_values(by=['created_at', 'fecha_actualizacion_pagos'], ascending=[False, True], na_position='first')
    
    # --- MODIFICACIÓN PARA PRUEBA RÁPIDA ---
    # df_ruts_a_consultar = df_ruts_a_consultar.head(10) # Descomentar para pruebas rápidas
    
    log_file.write(f"Se seleccionaron {len(df_ruts_a_consultar)} contactos para consultar su estado de pago.\n")
else:
    log_file.write("\n--- Bloque 4: Omitido por no haber datos de HubSpot ---\n")

In [ ]:
# ===================================================================
# CELDA 5: Consulta a API Externa y Cruce de Datos
# ===================================================================
df_cargainformacion = pd.DataFrame()
if not df_ruts_a_consultar.empty:
    log_file.write("\n--- Bloque 5: Consultando API externa y cruzando datos ---\n")
    resultados_rut = obtener_datos_desde_api_rut(df_ruts_a_consultar, columna_id='rut_sdv')
    if not resultados_rut.empty:
        resultados_rut = resultados_rut[resultados_rut['rut'].notna()]
        resultados_rut['rut_sdv'] = resultados_rut['rut'].astype(str)
        df_hubspot = df_ruts_a_consultar[['id', 'rut_sdv']].copy()
        df_cruce = pd.merge(resultados_rut, df_hubspot, on='rut_sdv', how='inner')
        df_cargainformacion = df_cruce.fillna('')
        log_file.write(f"Cruce de datos finalizado. Se encontraron {len(df_cargainformacion)} coincidencias.\n")
    else:
        log_file.write("La API externa no devolvió resultados.\n")
else:
    log_file.write("\n--- Bloque 5: Omitido por no haber contactos para consultar ---\n")

In [ ]:
# ===================================================================
# CELDA 6: Procesamiento de Datos de Contratos
# ===================================================================
df_expandido_final = pd.DataFrame()
if not df_cargainformacion.empty:
    log_file.write("\n--- Bloque 6: Procesando datos de contratos ---\n")
    df_cargainformacion = df_cargainformacion.dropna(subset=['contratos'])
    df_cargainformacion = df_cargainformacion[df_cargainformacion['contratos'].apply(lambda x: isinstance(x, list))]

    if not df_cargainformacion.empty:
        df_expandido = df_cargainformacion.explode('contratos')
        df_expandido = pd.concat([df_expandido.drop(columns=['contratos']), df_expandido['contratos'].apply(pd.Series)], axis=1)
        df_expandido = df_expandido.loc[:, ~df_expandido.columns.duplicated()]

        mapeo_estado_pago = {'AL DIA': 'Al día', 'LIBERADO': 'Liberado', 'LIBERADO DIRECTORIO FALMED': 'Liberado', 'FALMED SENIOR LIBERADO': 'Liberado', 'MOROSO12': 'Moroso', 'MOROSO' : 'Moroso'}
        df_expandido['descEstadoPago'] = df_expandido['descEstadoPago'].map(mapeo_estado_pago)

        mapeo_condicion_socio = {'AFILIADO': 'Afiliado', 'DESAFILIADO': 'Desafiliado', 'RENUNCIADO': 'Renunciado', 'REINSCRITO': 'Afiliado'}
        df_expandido['descCondicionSocio'] = df_expandido['descCondicionSocio'].map(mapeo_condicion_socio)

        df_expandido['idMedico'] = pd.to_numeric(df_expandido['idMedico'], errors='coerce').astype('Int64')
        df_expandido_final = df_expandido.drop_duplicates(subset=["idMedico", "descEstamento"], keep="last")
        df_expandido_final['fechaInscripcion'] = pd.to_datetime(df_expandido_final['fechaInscripcion'], format='%d-%m-%Y', errors='coerce')

        df_expandido_final = df_expandido_final.pivot(index='id', columns='descEstamento', values=["descCondicionSocio", "fechaInscripcion", "descEstadoPago"])
        df_expandido_final.columns = ['_'.join(map(str, col)).strip() for col in df_expandido_final.columns.values]
        df_expandido_final.reset_index(inplace=True)
        log_file.write(f"Se procesaron y pivotaron los datos de contratos para {len(df_expandido_final)} contactos.\n")
    else:
        log_file.write("No se encontraron contratos para procesar.\n")
else:
    log_file.write("\n--- Bloque 6: Omitido por no haber datos para procesar ---\n")

In [ ]:
# ===================================================================
# CELDA 7: Actualización Final de Contactos en HubSpot
# ===================================================================
if not df_expandido_final.empty:
    log_file.write("\n--- Bloque 7: Actualizando contactos en HubSpot ---\n")
    
    # Preparar columnas y rellenar nulos
    for col in ['descCondicionSocio_CLUB CAMPO', 'descEstadoPago_CLUB CAMPO', 'descCondicionSocio_FATMED', 'descEstadoPago_FATMED', 'descEstadoPago_FSG', 'descEstadoPago_FALMED']:
        if col not in df_expandido_final.columns:
            df_expandido_final[col] = 'SIN_AFILIACION'
    df_expandido_final.fillna('', inplace=True)

    total_a_actualizar = len(df_expandido_final)
    log_file.write(f"Iniciando la actualización de {total_a_actualizar} contactos...\n")
    
    headers = {'Content-Type': 'application/json', 'Authorization': f"Bearer {os.getenv('HUBSPOT_API_KEY')}"}

    for index, row in df_expandido_final.iterrows():
        contact_id = row['id']
        fecha_inscripcion = row.get('fechaInscripcion_COLEGIO', '')
        if pd.notna(fecha_inscripcion) and fecha_inscripcion != '':
             fecha_inscripcion = fecha_inscripcion.strftime('%Y-%m-%d')
        else:
            fecha_inscripcion = ''
        
        data = {
            "properties": {
                "fecha_inscripcion_al_colegio": fecha_inscripcion,
                "condicion": row.get('descCondicionSocio_COLEGIO', ""),
                "estado_pago": row.get('descEstadoPago_COLEGIO', ""),
                "club_medico": str(row.get('descCondicionSocio_CLUB CAMPO', "")).upper(),
                "estado_pago_club_medico": str(row.get('descEstadoPago_CLUB CAMPO', "")).upper().replace('AL DÍA', 'AL_DIA'),
                "fecha_actualizacion_pagos": datetime.today().strftime('%Y-%m-%d'),
                "estado_pago_fsg": limpiar_valor(row.get('descEstadoPago_FSG', "")),
                "estado_pago_falmed": limpiar_valor(row.get('descEstadoPago_FALMED', "")),
                "estado_pago_fatmed": limpiar_valor(row.get('descEstadoPago_FATMED', ""))
            }
        }

        url = f'https://api.hubapi.com/crm/v3/objects/contacts/{contact_id}'
        response = requests.patch(url, headers=headers, json=data)

        if response.status_code == 200:
            log_file.write(f"({index + 1}/{total_a_actualizar}) Éxito: Contacto {contact_id} actualizado.\n")
        else:
            log_file.write(f"!!! ERROR al actualizar {contact_id}: {response.status_code} - {response.text}\n")
        
        time.sleep(0.3) # Pausa para no saturar la API
else:
    log_file.write("\n--- Bloque 7: Omitido por no haber datos para actualizar ---\n")

In [ ]:
# ===================================================================
# CELDA FINAL: Cerrar el archivo de log
# ===================================================================
end_time = time.time()
mem_end = process.memory_info().rss
log_file.write("\n====================================================\n")
log_file.write(f"== FIN DEL SCRIPT (Duración: {end_time - start_time:.2f} segundos) ==\n")
log_file.write(f"== Memoria usada: {(mem_end - mem_start) / (1024 * 1024):.2f} MB ==\n")
log_file.write("====================================================\n")
log_file.close()